## Vector Search on Azure Cognitive Search via REST Endpoint

### Set environment variables

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

acs_key  = os.getenv("COGNITIVE_SEARCH_KEY")
if acs_key is None or acs_key == "":
    print("COGNITIVE_SEARCH_KEY environment variable not set.")
    exit()

aoai_key  = os.getenv("AZURE_OPENAI_KEY")
if aoai_key is None or aoai_key == "":
    print("AZURE_OPENAI_KEY environment variable not set.")
    exit()

acs_endpoint = 'https://cogsearch02.search.windows.net'
acs_index_definition = 'index_definition/index_definition_text.json'
acs_api_version = '2023-07-01-Preview'
aoai_endpoint = 'https://azure-openai-dnai.openai.azure.com'
aoai_api_version = '2023-08-01-preview'
aoai_embedding_deployed_model = 'embedding-ada'

### Helper methods

In [ ]:
import requests
import json

def search_vector_similarity(query_vector, top_doc_count, acs_endpoint, acs_index,acs_key, acs_api_version):
    url = f"{acs_endpoint}/indexes/{acs_index}/docs/search?api-version={acs_api_version}"

    headers = {
        "Content-Type": "application/json",
        "api-key": acs_key
    }

    request_body = {
        "vectors": [{
            "value": query_vector,
            "fields": "content_vector",
            "k": top_doc_count
        }],
        "select": "title, category"
    }
    request_body = json.dumps(request_body)
    response = requests.request('POST', url, headers=headers, data=request_body)

    docs = [(item['title'] , item['category']) for item in response.json()['value']]

    return docs

def get_acs_index_name(acs_index_definition):
    index_json_content = read_json_file(acs_index_definition)
    index_json = json.loads(index_json_content)
    index_name = index_json['name']

    return index_name

def read_json_file(file_path):
    with open(file_path, "r") as file:
        return file.read()

### Perform a vector similarity search

In [ ]:
query = 'tools that works without server'
index_name = "text-sample"
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

search_results = search_vector_similarity(query_vector, 5, acs_endpoint, index_name,acs_key, acs_api_version)

print(search_results)

### Simple vector similarity search

In [ ]:
query = 'tools for web hosting'
top_doc_count = 3
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)


url = f"{acs_endpoint}/indexes/{index_name}/docs/search?api-version={acs_api_version}"

headers = {
    "Content-Type": "application/json",
    "api-key": acs_key
}

request_body = {
    "vectors": [{
        "value": query_vector,
        "fields": "content_vector",
        "k": top_doc_count
    }],
    "select": "title, category"
}
request_body = json.dumps(request_body)
response = requests.request('POST', url, headers=headers, data=request_body)

docs = [(item['title'] , item['category']) for item in response.json()['value']]

print(docs)

### Cross column vector similarity search

In [ ]:
query = 'tools for web hosting'
top_doc_count = 3
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)


url = f"{acs_endpoint}/indexes/{index_name}/docs/search?api-version={acs_api_version}"

headers = {
    "Content-Type": "application/json",
    "api-key": acs_key
}

request_body = {
    "vectors": [{
        "value": query_vector,
        "fields": "title_vector",
        "k": top_doc_count
    },
    {
        "value": query_vector,
        "fields": "content_vector",
        "k": top_doc_count
    }
    ],
    "select": "title, category"
}
request_body = json.dumps(request_body)
response = requests.request('POST', url, headers=headers, data=request_body)

docs = [(item['title'] , item['category']) for item in response.json()['value']]

print(docs)

### Hybrid search

In [ ]:
query = 'tools that works without server'
top_doc_count = 3
query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)


url = f"{acs_endpoint}/indexes/{index_name}/docs/search?api-version={acs_api_version}"

headers = {
    "Content-Type": "application/json",
    "api-key": acs_key
}

request_body = {
    "vectors": [{
        "value": query_vector,
        "fields": "title_vector,content_vector",
        "k": top_doc_count
    }],
    "select": "title, category",
    "filter": "category eq 'Web'"
}
request_body = json.dumps(request_body)
response = requests.request('POST', url, headers=headers, data=request_body)

print(response.text)
docs = [(item['title'] , item['category']) for item in response.json()['value']]

print(docs)

### Perform a vector similarity search for docs

In [ ]:
query = 'when are performance review announced?'

query_vector = get_embedding(query, engine = aoai_embedding_deployed_model)

search_results = search_vector_similarity(query_vector, 5, acs_endpoint, index_name, acs_key, acs_api_version)

print(search_results)

### Perform a vector similarity search for image

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

query = 'flower with hand'

query_vector = vectorize_text_com_vision(com_vision_endpoint,com_vision_key,query)
search_results = search_vector_similarity(query_vector, 1, acs_endpoint, index_name,acs_key, acs_api_version)

show_image(image_folder, search_results[0])